In [1]:
%load_ext autoreload
%autoreload 2

from binance.client import Client
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from datetime import datetime
from libs import *
from utils import *

with open("C:\\Users\\thang\\OneDrive\\Documents\\binance_cres\\api_key.txt") as api_file:
    api_key = api_file.readline()
with open("C:\\Users\\thang\\OneDrive\\Documents\\binance_cres\\sec_key.txt") as api_file:
    api_secret = api_file.readline()
# make binance object
client = Client(api_key, api_secret)

# Parameters
target_coin = 'ETCUSDT'


In [2]:
## Raw data
# agg_trades = client.aggregate_trade_iter(symbol='ETCUSDT', start_str='60 minutes ago UTC')

# # iterate over the trade iterator
# df_dict = {'a':[], 'p':[], 'q':[], 'f':[], 'l':[],'T':[], 'm':[],'M':[]}
# for trade in agg_trades:
#     # prices.append(trade["p"])
#     df_dict['a'].append(trade["a"])
#     df_dict['p'].append(trade["p"])
#     df_dict['q'].append(trade["q"])
#     df_dict['f'].append(trade["f"])
#     df_dict['l'].append(trade["l"])
#     df_dict['T'].append(trade["T"])
#     df_dict['m'].append(trade["m"])
#     df_dict['M'].append(trade["M"])
#     print(trade)

# df = pd.DataFrame.from_dict(df_dict)
# interested_df = df[['p','q','T']]  # get price as p, quantity as q, Time as T

In [3]:
avg_data = client.get_klines(symbol=target_coin, interval=Client.KLINE_INTERVAL_30MINUTE)
cols = ['open_time',  # Open time
            'open',  # Open
            'high',  # High
            'low',  # Low
            'close',  # Close
            'vol',  # Volume
            'close_time',  # Close time
            'quote_ass_vol',  # Quote asset volume
            'no_trade',  # Number of trades
            'base_ass_vol',  # Taker buy base asset volume
            'qoute_ass_vol',  # Taker buy quote asset volume
            'ignore']   # Can be ignored
df = pd.DataFrame(avg_data, 
    columns = cols)

important_features_df = df[['close','close_time','vol']]  # get important features
important_features_df.close_time = important_features_df.close_time.apply(lambda x: x/1000)  # convert milisecond to second
important_features_df = data_converter(important_features_df)  # conver time data to sequence vector

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\Users\thang\Documents\pred_coin_price\ts_pred_viz\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Day sin'] = np.sin(date_time * (2 * np.pi / day))
c:\Users\thang\Documents\pred_coin_price\ts_pred_viz\utils.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [4]:
important_features_df.describe()

,close,vol,Day sin,Day cos,Year sin,Year cos
count,500.000000,5.000000e+02,5.000000e+02,5.000000e+02,500.000000,500.000000
mean,94.624048,2.011190e+05,2.220755e-02,-1.947549e-02,0.729669,-0.681843
std,14.727671,1.709804e+05,7.078198e-01,7.071923e-01,0.035336,0.037812
min,49.504000,4.307619e+04,-1.000000e+00,-1.000000e+00,0.666757,-0.745275
25%,88.028250,1.011704e+05,-7.071067e-01,-7.071068e-01,0.699401,-0.714730
50%,94.392500,1.517181e+05,7.272503e-08,-7.272329e-08,0.730646,-0.682756
75%,104.828000,2.315755e+05,7.071068e-01,7.071067e-01,0.760431,-0.649418
max,127.869000,1.664977e+06,1.000000e+00,1.000000e+00,0.788697,-0.614782


In [5]:
# data splitting
# column_indices = {name: i for i, name in enumerate(important_features_df.columns)}

n = len(important_features_df)
train_df = important_features_df[0:int(n*0.7)]  # 70% train data
val_df = important_features_df[int(n*0.7):int(n*0.9)]  # 20% val data
test_df = important_features_df[int(n*0.9):]  # 10% test data

num_features = important_features_df.shape[1]  # number of features

In [6]:
# normalization
train_mean = train_df.mean()
train_std = train_df.std()

# normalize base on train_data distribution
train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std


In [7]:
time_predictors = 24  # number of historical data as predictor
time_shift = 24  # number of timestamp to target
label_width = 1  # number of predicting timestamp

In [8]:
# def input_windows_maker(data_matrix):
#     inputs = data_matrix[:, :24, :]
#     return inputs

# def output_windows_maker(data_matrix):
#     inputs = data_matrix[:, 24:, :]
#     return inputs
# ds_output = ds.map(output_windows_maker)
# for k in ds_output:
#     print(k.numpy().shape)

In [9]:
# prepare data
single_step_window = WindowGenerator(
    input_width=time_predictors, label_width=label_width, shift=time_shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=['close'])
single_step_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [47]
Label column name(s): ['close']

In [10]:
count = 0
for element in single_step_window.train[0]:
    count += 1
    print(count)
    print(element.numpy())
    if len(element.numpy()) < 32:
        break
    if count == 1:
        train_input = element.numpy()
    else:
        train_input = np.append(train_input, element.numpy(), axis=0)

282e+00]
  ...
  [ 1.8567853e-01 -5.4505813e-01  3.1684849e-01 -1.3599648e+00
   -1.2720699e+00 -1.2505670e+00]
  [ 2.2807574e-02 -8.1901377e-01  1.3698444e-01 -1.3963134e+00
   -1.2824571e+00 -1.2600414e+00]
  [-9.7420804e-02 -6.7261457e-01 -4.6010718e-02 -1.4084990e+00
   -1.2928482e+00 -1.2695125e+00]]

 ...

 [[ 1.7570886e-01 -2.8484806e-01 -1.4359971e+00  2.0178506e-01
   -4.4356191e-01 -4.7230324e-01]
  [ 2.6099402e-01 -1.0191450e-01 -1.4002199e+00  3.8452116e-01
   -4.5363414e-01 -4.8204061e-01]
  [-2.2006616e-02  3.8446924e-01 -1.3412719e+00  5.6094950e-01
   -4.6371031e-01 -4.9177477e-01]
  ...
  [ 5.9226370e-01 -1.8906114e-01  1.3081982e+00  3.8452137e-01
   -6.5590465e-01 -6.7611504e-01]
  [ 5.3234690e-01 -2.6442084e-01  1.3439754e+00  2.0178527e-01
   -6.6605937e-01 -6.8578500e-01]
  [ 6.3747269e-01 -5.2770764e-01  1.3559695e+00  1.5868064e-02
   -6.7621797e-01 -6.9545180e-01]]

 [[-1.1301693e-01  6.1029172e+00 -1.2601615e+00  7.2805136e-01
    1.3878796e+00  1.4240048e+00]

In [11]:
train_input.shape

(288, 24, 6)

In [12]:
count = 0
for element in single_step_window.train[1]:
    count += 1
    print(count)
    print(element.numpy().shape)
    if len(element.numpy()) < 32:
        break
    if count == 1:
        train_label = element.numpy()
    else:
        train_label = np.append(train_label, element.numpy(), axis=0)

1
(32, 1, 1)
2
(32, 1, 1)
3
(32, 1, 1)
4
(32, 1, 1)
5
(32, 1, 1)
6
(32, 1, 1)
7
(32, 1, 1)
8
(32, 1, 1)
9
(32, 1, 1)
10
(15, 1, 1)


In [13]:
train_label.shape

(288, 1, 1)

In [14]:
### main model ###
# linear module
linear_input = tf.keras.Input(shape=(single_step_window.example[0].shape[1:]), batch_size=single_step_window.example[0].shape[0])  # batch 32 input shape = 24,6
x = tf.keras.layers.Flatten()(linear_input)
x = tf.keras.layers.Dense(units=64, activation='relu')(x)
x = tf.keras.layers.Dense(units=64, activation='relu')(x)
x = tf.keras.layers.Dense(units=1, activation='relu')(x)
linear_model = tf.keras.Model(inputs=linear_input, outputs=x)  # output = 32,1

# lstm module
lstm_input = tf.keras.Input(shape=(single_step_window.example[0].shape[1:]), batch_size=single_step_window.example[0].shape[0])
y = tf.keras.layers.LSTM(32, return_sequences=False)(lstm_input)
y = tf.keras.layers.Dense(units=1, activation='relu')(y)
lstm_model = tf.keras.Model(inputs=lstm_input, outputs=y)  # output = 32,1

# intersection
z = tf.keras.layers.concatenate([linear_model.output, lstm_model.output])
z = tf.keras.layers.Dense(units=64, activation='relu')(z)
z = tf.keras.layers.Dense(units=1, activation='relu')(z)
comb_model = tf.keras.Model(inputs=[linear_input, lstm_input], outputs=[z])


# training setting

MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [15]:
# tf.keras.utils.plot_model(comb_model, "multi_input_and_output_model.png", show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')
comb_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
comb_model.summary()
comb_model.fit([train_input, train_input], [train_label], epochs=MAX_EPOCHS )

# define val data

In [ ]:
history = compile_and_fit(comb_model, single_step_window)
# linear.summary()
val_performance = {}
performance = {}
val_performance['Linear'] = comb_model.evaluate(single_step_window.val)
performance['Linear'] = comb_model.evaluate(single_step_window.test, verbose=0)

In [ ]:
single_step_window.plot(linear, max_subplots=2)

In [ ]:
linear.summary()

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=lstm_model.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', linear(wide_window.example[0]).shape)